#### Set environments

In [1]:
import torch, os, gc
import torch.nn as nn
import pandas as pd
import numpy as np
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from torch.nn.utils.rnn import pack_padded_sequence
from sklearn.metrics import confusion_matrix, matthews_corrcoef, accuracy_score,\
f1_score, precision_score, recall_score, roc_auc_score, average_precision_score

In [2]:
# Set options
embed_ver = ["cnn", "lstm", "clstm"]

data_path = "../data/"
model_path = "../model/seq_gen/"
result_path = "../result/prd-seq_gen/"
os.makedirs(result_path, exist_ok=True)

# data preprocessing options
col_str = ['file_id', 'organism', 'locus_tag', 'ess']

# model options
layer_num = 2
max_len = 1600
batch_size = 256

In [3]:
# Set data list for test dataset
ts_data = {
    "data1": ["C018"],  # "Escherichia coli K-12 BW25113"
    "data2": ["C016"],  # "Escherichia coli K-12 MG1655"
    "data3": ["O046"],  # "synthetic bacterium JCVI-Syn3A"
    "data4": ["C048"],  # Bacteroides thetaiotaomicron VPI-5482
    "data5": ["C050"]  # Salmonella enterica subsp. enterica serovar Typhimurium str. 14028S
}

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
# Define function to record perfomance result
def record_perform(emb_ver, file_id, organ, y_real, y_conf, y_prd):
    y_real = y_real.cpu().numpy()
    y_conf = y_conf.cpu().numpy()
    y_prd = y_prd.cpu().numpy()
    
    if file_id != "O046":
        auc_roc = [roc_auc_score(y_real, y_conf)]
        auc_pr = [average_precision_score(y_real, y_conf)]
    else:
        auc_roc = None
        auc_pr = None
    
    tn, fp, fn, tp = confusion_matrix(y_real, y_prd).ravel()
    
    result = pd.DataFrame({
        "embed": [emb_ver],
        "file": [file_id],
        "organism": [organ],
        "tp": [tp],
        "fp": [fp],
        "tn": [tn],
        "fn": [fn],
        "mcc": [matthews_corrcoef(y_real, y_prd)],
        "acc": [accuracy_score(y_real, y_prd)],
        "f1": [f1_score(y_real, y_prd)],
        "prc": [precision_score(y_real, y_prd)],
        "rec": [recall_score(y_real, y_prd)],
        "npv": [precision_score(1 - y_real, 1 - y_prd)],
        "tnr": [recall_score(1 - y_real, 1 - y_prd)],
        "auc-roc": auc_roc,
        "auc-pr": auc_pr
    })

    return result


In [6]:
# Set model architecture
class ClassifierCNN(nn.Module):
    def __init__(self, num_layer, pool_len, vocab_size, max_len):
        super(ClassifierCNN, self).__init__()
        emb_dim = 16
        self.embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=emb_dim, padding_idx=0)

        in_dim = emb_dim
        out_dim = 1024
        layers = []
        for i in range(num_layer):
            layers.append(nn.Conv1d(in_dim, out_dim, kernel_size=8))
            self.initialize_weights(layers[-1])
            layers.append(nn.GELU())
            layers.append(nn.AdaptiveMaxPool1d(max(pool_len, max_len // (8 * (i + 1)))))
            in_dim = out_dim
        layers.append(nn.AdaptiveMaxPool1d(pool_len))
        self.emb_block = nn.Sequential(*layers)
        
        self.bn = nn.BatchNorm1d(out_dim)
        self.do = nn.Dropout(0.5)
        self.fc = nn.Linear(out_dim, 1)
    
    def initialize_weights(self, layer):
        nn.init.kaiming_normal_(layer.weight, mode='fan_in', nonlinearity='linear')
        if layer.bias is not None:
            nn.init.zeros_(layer.bias)

    def forward(self, x):
        # embedding
        emb = self.embedding(x)
        emb = emb.permute(0, 2, 1)
        emb = self.emb_block(emb)
        # # global average pooling w/o paddings
        # val_lens = (emb != 0).any(dim=1).sum(dim=1, keepdim=True)
        # emb = emb.sum(dim=2) / val_lens
        emb = emb.mean(dim=2)
        emb = self.bn(emb)
        # classification
        x = self.do(emb)
        x = self.fc(x)
        return x.squeeze(1)

In [7]:
# Set model architecture
class ClassifierLSTM(nn.Module):
    def __init__(self, num_layer, input_len, vocab_size):
        super(ClassifierLSTM, self).__init__()
        self.input_len = input_len
        
        emb_dim = 16
        self.embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=emb_dim, padding_idx=0)
        
        out_dim = 1024
        self.lstm = nn.LSTM(emb_dim, out_dim // 2, num_layers=num_layer, bidirectional=True, batch_first=True)
        
        self.bn = nn.BatchNorm1d(out_dim)
        self.do = nn.Dropout(0.5)
        self.fc = nn.Linear(out_dim, 1)
    
    def initialize_weights(self, layer):
        nn.init.kaiming_normal_(layer.weight, mode='fan_in', nonlinearity='linear')
        if layer.bias is not None:
            nn.init.zeros_(layer.bias)

    def forward(self, x):
        # extract valid input sequences
        val_lens = (x != 0).sum(dim=1)
        start_idx = torch.clamp((val_lens - self.input_len) // 2, min=0)
        x = torch.stack([x[i, start:start+self.input_len] for i, start in enumerate(start_idx)], dim=0)
        # embedding
        emb = self.embedding(x)
        val_lens = (x != 0).sum(dim=1).cpu()
        emb = pack_padded_sequence(emb, val_lens, batch_first=True, enforce_sorted=False)
        out, (hidden, cell) = self.lstm(emb)
        hidden = hidden.view(self.lstm.num_layers, 2, x.size(0), self.lstm.hidden_size)
        emb = torch.cat((hidden[-1, 0, :, :], hidden[-1, 1, :, :]), dim=1)
        emb = self.bn(emb)
        # classification
        x = self.do(emb)
        x = self.fc(x)
        return x.squeeze(1)

In [8]:
# Set model architecture
class ClassifierCNN_LSTM(nn.Module):
    def __init__(self, pool_len, vocab_size):
        super(ClassifierCNN_LSTM, self).__init__()
        emb_dim = 16        
        self.embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=emb_dim, padding_idx=0)

        out_dim = 1024
        self.conv1d = nn.Conv1d(emb_dim, out_dim, kernel_size=8)
        self.initialize_weights(self.conv1d)
        self.pool = nn.AdaptiveAvgPool1d(pool_len)
        self.lstm = nn.LSTM(out_dim, out_dim // 2, bidirectional=True, batch_first=True)
        
        self.bn = nn.BatchNorm1d(out_dim)
        self.do = nn.Dropout(0.5)
        self.fc = nn.Linear(out_dim, 1) 
    
    def initialize_weights(self, layer):
        nn.init.kaiming_normal_(layer.weight, mode='fan_in', nonlinearity='linear')
        if layer.bias is not None:
            nn.init.zeros_(layer.bias)

    def forward(self, x):
        # embedding
        emb = self.embedding(x)
        emb = emb.permute(0, 2, 1)
        emb = F.gelu(self.conv1d(emb))
        emb = self.pool(emb)
        emb = emb.permute(0, 2, 1)
        # mark valid feature steps
        val_lens = (emb != 0).any(dim=2).sum(dim=1).cpu()
        emb = pack_padded_sequence(emb, val_lens, batch_first=True, enforce_sorted=False)
        out, (hidden, cell) = self.lstm(emb)
        emb = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1)
        emb = self.bn(emb)
        # classification
        x = self.do(emb)
        x = self.fc(x)
        return x.squeeze(1)

#### Prepare data

In [ ]:
# Load dataset
df = pd.read_csv(data_path + f"data-seq_raw-ts.csv")
display(df)

In [ ]:
# Set vocabulary dictionary
valid_aa = 'ACDEFGHIKLMNPQRSTVWY'
aa_to_int = {aa: idx + 1 for idx, aa in enumerate(valid_aa)}  # 0 is padding

print(aa_to_int)

In [ ]:
# Replace invalid residues
df['aa_seq'] = df['aa_seq'].str.replace(f"[^{valid_aa}]", "", regex=True)

# # Filter invalid sequences
# df = df[df['aa_seq'].apply(lambda seq: set(seq).issubset(set(valid_aa)))]
# df = df.reset_index(drop=True)

print(df.shape)

In [ ]:
# Truncate sequences to max length
seq = df['aa_seq'].str[:max_len].to_list()

print(len(seq))
print(seq[:2])

In [ ]:
# Set integer encoding function
def int_encode_seq(seq, aa_to_int):
    seq_list = list(seq)
    return [aa_to_int[aa] for aa in seq_list]

# Encode sequences to integers
encoded_seq = [int_encode_seq(s, aa_to_int) for s in seq]

print(len(encoded_seq))
print(encoded_seq[:2])

In [ ]:
# Set padding function
def pad_seq(seq, max_len):
    return seq + [0] * (max_len - len(seq))

# Pad sequences
encoded_seq = np.array([pad_seq(s, max_len) for s in encoded_seq])

print(encoded_seq.shape)
print(encoded_seq[:2])

In [ ]:
data = pd.concat([df[col_str], pd.DataFrame(encoded_seq)], axis=1)

display(data)

In [16]:
col_num = [col for col in data.columns if col not in col_str]

In [ ]:
# get test datasets
loc_ts = {}
data_ts = {}
org_ts = {}
for ts_ver, ids in ts_data.items():
    # get test sample locations
    loc_ts[ts_ver] = data['file_id'].isin(ids)
    # get test samples
    data_ts[ts_ver] = data[loc_ts[ts_ver]]
    org = []
    # get test organism list
    for i in ids:
        organ = data_ts[ts_ver]['organism'][data_ts[ts_ver]['file_id'] == i].to_list()
        if len(organ) > 0:
            org.append(organ[0])
    org_ts[ts_ver] = org

    print("Test dataset(" + ts_ver + "):", data_ts[ts_ver].shape)
print("Test organism:", org_ts, len(org_ts))

In [ ]:
# split info.& inputs & labels of the test datasets
info_ts = {}
y_ts = {}
test_loader = {}
for ts_ver, df in data_ts.items():
    info_ts[ts_ver] = df[col_str]
    X_ts = torch.tensor(df[col_num].astype('long').values)
    y_ts[ts_ver] = torch.tensor(df['ess'].astype('long').values)
    print("Splited test dataset(" + ts_ver + "):", X_ts.shape, y_ts[ts_ver].shape)                    
    # generate dataloader by the test datasets
    dataset_ts = TensorDataset(X_ts, y_ts[ts_ver])
    test_loader[ts_ver] = DataLoader(dataset_ts, batch_size=256, shuffle=False)

In [ ]:
# get the total test dataset
loc_ts_all = [sum(loc) >= 1 for loc in zip(*loc_ts.values())]
info_ts_all = data.loc[loc_ts_all, col_str]
X_ts_all = torch.tensor(data.loc[loc_ts_all, col_num].astype('long').values)
y_ts_all = torch.tensor(data.loc[loc_ts_all, 'ess'].astype('long').values)

print("Splited test dataset(all):", X_ts_all.shape, y_ts_all.shape)

In [20]:
# generate dataloader of total test dataset
test_all_dataset = TensorDataset(X_ts_all, y_ts_all)
test_all_loader = DataLoader(test_all_dataset, batch_size=256, shuffle=False)

#### Evaluate model

In [21]:
df_eval = pd.DataFrame()

for ver in embed_ver:
    #### Evaluate model ####
    # set model name
    model_name = f"seq_gen-{ver}"
    print(f"\n===== Test model: {model_name} ====")
    
    if ver == 'cnn':
        model = ClassifierCNN(num_layer=layer_num, pool_len=25, vocab_size=len(aa_to_int) + 1, max_len=max_len).to(device)
        model.load_state_dict(torch.load(model_path + model_name + ".pt", map_location=device))
    elif ver == 'lstm':
        model = ClassifierLSTM(num_layer=layer_num, input_len=100, vocab_size=len(aa_to_int) + 1).to(device)
        model.load_state_dict(torch.load(model_path + model_name + ".pt", map_location=device))
    else:
        model = ClassifierCNN_LSTM(pool_len=25, vocab_size=len(aa_to_int) + 1).to(device)
        model.load_state_dict(torch.load(model_path + model_name + ".pt", map_location=device))
    
    model.eval()

    ## model evaluations by test dataset ##
    for ts_ver, ids in ts_data.items():
        all_preds = []
        all_labels = []
        with torch.no_grad():
            for X_batch, y_batch in test_loader[ts_ver]:
                X_batch = X_batch.to(device)
                y_batch = y_batch.to(device)
                preds = model(X_batch).squeeze()
                all_preds.append(preds.cpu())
                all_labels.append(y_batch.cpu())
        
        # concatenate results to one tensor
        all_preds = torch.cat(all_preds)
        all_labels = torch.cat(all_labels)

        # convert logits to confidences & classes
        prd_conf = torch.sigmoid(all_preds)
        prd_cls = (prd_conf >= 0.5).int()
        # performances by testset
        perform = record_perform(
            emb_ver=ver,
            file_id="+".join(ids),
            organ="+".join(org_ts[ts_ver]),
            y_real=y_ts[ts_ver],
            y_conf=prd_conf,
            y_prd=prd_cls,
        )
        display(perform)
        df_eval = pd.concat([df_eval, perform], ignore_index=True)
    
    
    ## model evaluation on the total test dataset ##
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for X_batch, y_batch in test_all_loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)
            preds = model(X_batch).squeeze()
            all_preds.append(preds.cpu())
            all_labels.append(y_batch.cpu())
    
    # concatenate results to one tensor
    all_preds = torch.cat(all_preds)
    all_labels = torch.cat(all_labels)

    # convert logits to confidences & classes
    prd_conf = torch.sigmoid(all_preds)
    prd_cls = (prd_conf >= 0.5).int()

    # performances on total testset
    perform = record_perform(
        emb_ver=ver,
        file_id="+".join([i for ids in ts_data.values() for i in ids]),
        organ="+".join([org for orgs in org_ts.values() for org in orgs]),
        y_real=y_ts_all,
        y_conf=prd_conf,
        y_prd=prd_cls
    )
    display(perform)
    df_eval = pd.concat([df_eval, perform], ignore_index=True)

    # concatenate the protein info. & predicted confidences
    df_prd = pd.DataFrame(prd_conf, columns=["conf"], index=info_ts_all.index)
    df_prd = pd.concat([info_ts_all, df_prd], axis=1)

    # save the model prediction result
    df_prd.to_csv(result_path + f"prd-{model_name}.csv", index=False)

    gc.collect()

# save the model perfomance result
display("Model performance:", df_eval)
df_eval.to_csv("../result/eval-seq_gen.csv", index=False)


===== Test model: seq_gen-cnn-10 ====


,embed,pool,testset,file,organism,tp,fp,tn,fn,mcc,acc,f1,prc,rec,npv,tnr,auc-roc,auc-pr
0,cnn,10,data1,C018,Escherichia coli K-12 BW25113,223,292,3723,75,0.528392,0.914908,0.548585,0.43301,0.748322,0.980253,0.927273,0.896913,0.565699


,embed,pool,testset,file,organism,tp,fp,tn,fn,mcc,acc,f1,prc,rec,npv,tnr,auc-roc,auc-pr
0,cnn,10,data2,C016,Escherichia coli K-12 MG1655,234,288,3735,56,0.564588,0.920241,0.576355,0.448276,0.806897,0.985228,0.928412,0.932575,0.635496


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


,embed,pool,testset,file,organism,tp,fp,tn,fn,mcc,acc,f1,prc,rec,npv,tnr,auc-roc,auc-pr
0,cnn,10,data3,O046,synthetic bacterium JCVI-Syn3A,98,0,0,360,0.0,0.213974,0.352518,1.0,0.213974,0.0,0.0,None,None


...

,embed,pool,testset,file,organism,tp,fp,tn,fn,mcc,acc,f1,prc,rec,npv,tnr,auc-roc,auc-pr
0,clstm,100,data4,C048,Bacteroides thetaiotaomicron VPI-5482,299,178,4322,26,0.739343,0.95772,0.745636,0.626834,0.92,0.99402,0.960444,0.978639,0.776172


,embed,pool,testset,file,organism,tp,fp,tn,fn,mcc,acc,f1,prc,rec,npv,tnr,auc-roc,auc-pr
0,clstm,100,data5,C050,Salmonella enterica subsp. enterica serovar Ty...,84,529,4832,29,0.290682,0.898064,0.231405,0.137031,0.743363,0.994034,0.901324,0.887805,0.117733


,embed,pool,testset,file,organism,tp,fp,tn,fn,mcc,acc,f1,prc,rec,npv,tnr,auc-roc,auc-pr
0,clstm,100,test_all,C018+C016+O046+C048+C050,Escherichia coli K-12 BW25113+Escherichia coli...,1008,1369,16530,476,0.488608,0.904813,0.522145,0.424064,0.679245,0.97201,0.923515,0.8691,0.425139


'Model performance:'

,embed,pool,testset,file,organism,tp,fp,tn,fn,mcc,acc,f1,prc,rec,npv,tnr,auc-roc,auc-pr
0,cnn,10,data1,C018,Escherichia coli K-12 BW25113,223,292,3723,75,0.528392,0.914908,0.548585,0.433010,0.748322,0.980253,0.927273,0.896913,0.565699
1,cnn,10,data2,C016,Escherichia coli K-12 MG1655,234,288,3735,56,0.564588,0.920241,0.576355,0.448276,0.806897,0.985228,0.928412,0.932575,0.635496
2,cnn,10,data3,O046,synthetic bacterium JCVI-Syn3A,98,0,0,360,0.000000,0.213974,0.352518,1.000000,0.213974,0.000000,0.000000,NaN,NaN
3,cnn,10,data4,C048,Bacteroides thetaiotaomicron VPI-5482,295,169,4331,30,0.739747,0.958756,0.747782,0.635776,0.907692,0.993121,0.962444,0.974694,0.826376
4,cnn,10,data5,C050,Salmonella enterica subsp. enterica serovar Ty...,75,501,4860,38,0.264252,0.901535,0.217707,0.130208,0.663717,0.992242,0.906547,0.875880,0.113041
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,clstm,100,data2,C016,Escherichia coli K-12 MG1655,263,330,3693,27,0.599874,0.917227,0.595696,0.443508,0.906897,0.992742,0.917972,0.950882,0.571837
68,clstm,100,data3,O046,synthetic bacterium JCVI-Syn3A,113,0,0,345,0.000000,0.246725,0.395797,1.000000,0.246725,0.000000,0.000000,NaN,NaN
69,clstm,100,data4,C048,Bacteroides thetaiotaomicron VPI-5482,299,178,4322,26,0.739343,0.957720,0.745636,0.626834,0.920000,0.994020,0.960444,0.978639,0.776172
70,clstm,100,data5,C050,Salmonella enterica subsp. enterica serovar Ty...,84,529,4832,29,0.290682,0.898064,0.231405,0.137031,0.743363,0.994034,0.901324,0.887805,0.117733
